# WESAD Validation Notebook for FLIRT


In [1]:
# Import Packages
import pandas as pd
import numpy as np

import matplotlib; matplotlib.use('agg')
import matplotlib.pyplot as plt

import multiprocessing
from joblib import Parallel, delayed
from tqdm.autonotebook import trange

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn import utils, model_selection, metrics

from datetime import datetime, timedelta, timezone
from typing import List
import lightgbm as lgb
import glob2
import os 

import sys
sys.path.insert(1, '/home/fefespinola/ETHZ_Fall_2020/flirt-1')
import flirt.simple

In [2]:
The following function retrieves all HRV, EDA and ACC features per subject using the FLIRT pipeline


SyntaxError: invalid syntax (<ipython-input-2-c9c2e5945b79>, line 1)

In [2]:
def get_features_per_subject(path, window_length):
    features = flirt.simple.get_features_for_empatica_archive(zip_file_path = path,
                                      window_length = window_length,
                                      window_step_size = 0.25,
                                      hrv_features = False,
                                      eda_features = True,
                                      acc_features = False,
                                      bvp_features = False,
                                      temp_features = False,
                                      debug = True)
    return features

The following function determines the time offsets of the start and end of each relevant analysis period (baseline, stress, amusement). These offsets are combined with the timestamp stating the start of recording, to determine the absolute timestamps of the sections of interest for each subject. 

In [3]:
def find_label_timestamps(csv_path, StartingTime):

    ID = csv_path.split('/', 3)[2]
    df_timestamp = pd.read_csv(glob2.glob('project_data/WESAD/' + ID + '/*quest.csv')[0], delimiter = ';', header = 1).iloc[:2, :].dropna(axis = 1)
    print('===================================')
    print('Printing the timestamp for {0}'.format(ID))
    print('===================================')
    print(df_timestamp.head())
    
    # Start/End of experiment periods
    print('\nStart of the baseline: ' + str(df_timestamp['Base'][0]))
    print('End of the baseline: ' + str(df_timestamp['Base'][1]))
    print('Start of the fun: ' + str(df_timestamp['Fun'][0]))
    print('End of the fun: ' + str(df_timestamp['Fun'][1]))
    print('Start of the stress: ' + str(df_timestamp['TSST'][0]))
    print('End of the stress: ' + str(df_timestamp['TSST'][1]))
    
    # Get start and end time and assign label into a dict
    lab_dict = {'Base':0, 'TSST':1, 'Fun':2}
    labels_times_dict = {}
    for mode in df_timestamp.columns.tolist():
        print('mode', mode)
        if mode=='Base' or mode=='Fun' or mode=='TSST':
            labels_times_dict[mode] = [StartingTime + timedelta(minutes = int(str(df_timestamp[mode][0]).split(".")[0]))+ timedelta                                         (seconds = int(str(df_timestamp[mode][0]).split(".")[1])), 
                                    StartingTime + timedelta(minutes = int(str(df_timestamp[mode][1]).split(".")[0])) + timedelta                                           (seconds = int(str(df_timestamp[mode][1]).split(".")[1])), lab_dict[mode]]
            
            #labels_times_dict[mode] = [StartingTime + timedelta(minutes = float(df_timestamp[mode][0])), 
                                  #StartingTime + timedelta(minutes = float(df_timestamp[mode][1])), lab_dict[mode]]
        
    return labels_times_dict

In [4]:
import ast
from dateutil.parser import parse

def find_label_start_time(csv_path):
    ID = csv_path.split('/', 3)[2]
    timestamp = open(glob2.glob('project_data/WESAD/' + ID + '/*respiban.txt')[0], "r")
    for i in range(2):
        line = (timestamp.readline())
        line = line.strip()[2:]
        if i==1:
            dict = ast.literal_eval(line)
            start_time_str = dict['00:07:80:D8:AB:58']['time']
            date_str = dict['00:07:80:D8:AB:58']['date']
            datetime_str = date_str + " " + start_time_str
            #print(datetime_str)
            date_time_obj = parse(datetime_str)
            #print(date_time_obj)
            start_time = date_time_obj
            utc_time = start_time - timedelta(hours=2)

    timestamp.close()

    #df_timestamp = pd.read_table(glob2.glob('project_data/WESAD/' + ID + '/*respiban.txt')[0], delim_whitespace=True)#.iloc[:2, :].dropna(axis = 1)
    print('===================================')
    print('Printing the timestamp for {0}'.format(ID))
    print('===================================')
    #print(df_timestamp.head())
    return utc_time

Plots the training and validation classification metric evolution with the number of iterations. 

In [5]:
os.chdir('/home/fefespinola/ETHZ_Fall_2020/') #local directory where the script is
File_Path = glob2.glob('project_data/WESAD/**/*_readme.txt', recursive=True)
for subject_path in File_Path:
    start_time = find_label_start_time(subject_path)
    print(start_time)

Printing the timestamp for S10
2017-07-25 07:25:01
Printing the timestamp for S11
2017-07-25 11:33:01
Printing the timestamp for S13
2017-08-08 11:33:01
Printing the timestamp for S14
2017-08-09 07:31:01
Printing the timestamp for S15
2017-08-10 07:30:01
Printing the timestamp for S16
2017-08-10 12:21:01
Printing the timestamp for S17
2017-08-11 07:39:01
Printing the timestamp for S2
2017-05-22 07:39:01
Printing the timestamp for S3
2017-05-24 11:26:01
Printing the timestamp for S4
2017-06-13 08:57:01
Printing the timestamp for S5
2017-06-13 12:42:01
Printing the timestamp for S6
2017-06-14 11:40:01
Printing the timestamp for S7
2017-07-06 11:30:01
Printing the timestamp for S8
2017-07-10 11:28:01
Printing the timestamp for S9
2017-07-11 11:26:01


In [6]:
def render_metric(eval_results, metric_name):
    ax = lgb.plot_metric(evals_result, metric=metric_name, figsize=(10, 5))
    #plt.show()
    plt.savefig('/home/fefespinola/ETHZ_Fall_2020/plots/render_metric_all_ekf_feat.png')

Plots the 10 top important classification features, i.e. the ones that influence the output the most.

In [7]:
def render_plot_importance(gbm, importance_type, max_features=10, ignore_zero=True, precision=3):
    ax = lgb.plot_importance(gbm, importance_type=importance_type,
                             max_num_features=max_features,
                             ignore_zero=ignore_zero, figsize=(12, 8),
                             precision=precision)
    #plt.show()
    plt.savefig('/home/fefespinola/ETHZ_Fall_2020/plots/feature_importance_all_ekf_feat.png')


Main function that calls the above functions, determines the relevant data to use (i.e. that within the useful recording periods of baseline, stress and amusement) using the timestampp offsets, assignes the appropriate label to each sample and returns the full data with training samples and the corresponding labels.

In [8]:
def main():
    os.chdir('/home/fefespinola/ETHZ_Fall_2020/') #local directory where the script is
    df_all = pd.DataFrame(None)
    #relevant_features = pd.DataFrame(None)
    File_Path = glob2.glob('project_data/WESAD/**/*_readme.txt', recursive=True)
    window_length = 60 # in seconds
    window_shift = 0.25 # in seconds
    for subject_path in File_Path:
        print(subject_path)
        print(subject_path.split('/', 3)[2])
        ID = subject_path.split('/', 3)[2]
        zip_path = glob2.glob('project_data/WESAD/' + ID + '/*_Data.zip')[0]
        print(zip_path)
        features = get_features_per_subject(zip_path, window_length)
        features.index.name = 'timedata'
        E4Time = features.index[0]
        print(E4Time)
        StartingTime = find_label_start_time(subject_path)
        print(StartingTime)
        labels_times = find_label_timestamps(subject_path, StartingTime)
        #features.index.tz_localize(tz='UTC')
        relevant_features = features.loc[
            ((features.index.tz_localize(tz=None) >= labels_times['Base'][0]) & (features.index.tz_localize(tz=None) <= labels_times['Base'][1])) 
            | ((features.index.tz_localize(tz=None) >= labels_times['Fun'][0]) & (features.index.tz_localize(tz=None) <= labels_times['Fun'][1])) 
            | ((features.index.tz_localize(tz=None) >= labels_times['TSST'][0]) & (features.index.tz_localize(tz=None) <= labels_times['TSST'][1]))]

        relevant_features.insert(0, 'ID', ID)
        relevant_features['label'] = np.zeros(len(relevant_features))
        relevant_features.loc[(relevant_features.index.tz_localize(tz=None)>=labels_times['Fun'][0]) &
                                (relevant_features.index.tz_localize(tz=None)<=labels_times['Fun'][1]), 'label'] = labels_times['Fun'][2]
        relevant_features.loc[(relevant_features.index.tz_localize(tz=None)>=labels_times['TSST'][0]) & 
                            (relevant_features.index.tz_localize(tz=None)<=labels_times['TSST'][1]), 'label'] = labels_times['TSST'][2]

        # concatenate all subjects and add IDs
        df_all = pd.concat((df_all, relevant_features))
    
    print(df_all)

    return df_all

This function generates and saves feature matrices for the individual physiological signals 

In [9]:
def __get_subset_features(df_all, feature_name: str, eda_method:str='lpf'):
    
    if feature_name=='physio':
        small_df = df_all.loc[:, df_all.columns.str.startswith('hrv')&df_all.columns.str.startswith   ('eda')&df_all.columns.str.startswith('bvp')&df_all.columns.str.startswith('temp')]
        filename = 'features_all_' + features_name +'_' + eda_method + '_feat.csv'
    else:
        small_df = df_all.loc[:, df_all.columns.str.startswith(feature_name)]
        if feature_name=='eda':
            filename = 'features_all_' + features_name +'_' + eda_method + '_feat.csv'
        else:
            filename = 'features_all_' + features_name + '_feat.csv'
    small_df.to_csv(filename)


The following function retrieves the correct training and testing data for LOSO cross-validation. It also deals with missing data (inf and nan), and scales the features.

In [10]:
def __get_train_valid_data(df_all, cv_subject):
    scaler = StandardScaler()

    #training data
    X_train = df_all.loc[df_all['ID']!=cv_subject]  # 500 entities, each contains 10 features
    X_train = X_train.iloc[:, 1:len(df_all.columns)-1]
    X_train = X_train.replace([np.nan, np.inf, -np.inf], -1000)
    X_train = scaler.fit_transform(X_train)
    y_train = df_all.loc[df_all['ID']!=cv_subject, ['label']]  # binary target
    train_data = lgb.Dataset(X_train, label=y_train)

    #validation data
    X_test = df_all.loc[df_all['ID']==cv_subject]  # 500 entities, each contains 10 features
    X_test = X_test.iloc[:, 1:len(df_all.columns)-1]
    X_test = X_test.replace([np.nan, np.inf, -np.inf], -1000)
    X_test = scaler.transform(X_test)
    y_test = df_all.loc[df_all['ID']==cv_subject, ['label']]   # binary target
    test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

    return X_train, y_train, train_data, X_test, y_test, test_data
 

Run the evaluation script to retrieve the labeled data and train classifier to output f1-score

In [11]:
if __name__ == '__main__':
    df_all = main()
    df_all.to_csv('/home/fefespinola/ETHZ_Fall_2020/features_all_eda_default.csv')
    df_all = pd.read_csv('/home/fefespinola/ETHZ_Fall_2020/features_all_eda_default.csv')
    df_all.set_index('timedata', inplace=True)
    #df_all = df_all.loc[:, ~df_all.columns.str.startswith('hrv')]
    print(df_all)
    ID=df_all.ID
    print(ID.unique())

    ### for binary classification uncomment line below 
    #df_all['label'].replace(2, 0, inplace=True)
    #print('===== BINARY ====')

    print('---start classification---')

    df_all.round(4)
    df = df_all.replace([np.inf, -np.inf], np.nan) # np.inf leads to problems with some techniques

    # Clean columns that contain a lot of nan values 
    print(len(df), len(df.columns))
    df = df.dropna(axis=1, thresh=int(len(df)*0.8))
    print(len(df), len(df.columns))
    print('Columns dropped: ', df_all.drop(df.columns, axis=1).columns.values)

    stats = []

    cv = model_selection.LeaveOneGroupOut()

    X = df.drop(columns=['label', 'ID'])
    y = df['label'].astype('int')
    groups = df['ID']
    print("running %d-fold CV..." % (cv.get_n_splits(X, y, groups)))

    mean_fpr = np.linspace(0, 1, 100)
    fig, ax = plt.subplots(figsize=(15, 15))

    for train_index, test_index in cv.split(X, y, groups):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        #### for binary classification uncomment line below
        #params = {'objective': 'binary', 'is_unbalance': True}
        params = {'objective': 'multiclass', 'metric': 'multi_logloss', 'num_class':3,  'is_unbalance': True}
        model = lgb.LGBMClassifier(**params)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        stats.append({
            'f1': metrics.f1_score(y_test, y_pred, average="macro")})
            
        #print(metrics.classification_report(y_test, y_pred))
        print(confusion_matrix(y_test, y_pred))

    stats = pd.DataFrame(stats)
    print(stats.f1.mean())

    '''
    #parameters
    param = {'metric': 'auc_mu', 'learning_rate': 0.01, 'num_leaves': 31, 'is_unbalance':True,
        'verbose': 1, 'objective':'multiclass', 'num_class':3, 'lambda_l1':0, 'force_col_wise':True}
    
    subjects = ['S2', 'S3', 'S4', 'S10', 'S11', 'S13', 'S14', 'S15', 'S16', 'S17']

    ##### Start Classification
    f1_tot = 0
    f1_dict = {}
    #data with validation set
    for subj in subjects:
        # get data
        print('===Training for LOSO ', subj, '===')
        _, _, train_data, X_test, y_test, test_data = __get_train_valid_data(df_all, subj)
    
        evals_result = {}  # to record eval results for plotting
        
        #train normally
        bst_norm = lgb.train(param, train_data, num_boost_round=550, valid_sets=[train_data, test_data],                        evals_result=evals_result, verbose_eval=10)

        best_iteration = np.argmax(evals_result['valid_1']['auc_mu'][2:]) + 1
        print ('best iter', best_iteration)
        y_pred = bst_norm.predict(X_test, num_iteration=best_iteration)
        y_pred = np.argmax(y_pred, axis=1)

        f1_metric = f1_score(y_test, y_pred, average='macro')
        f1_dict[subj] = f1_metric
        print(f1_dict)
        f1_tot = f1_tot + f1_metric
        print(confusion_matrix(y_test, y_pred))


    f1_tot = f1_tot/len(subjects)
    print(f1_tot)
    #render_metric(evals_result, param['metric'])
    #render_plot_importance(bst_norm, importance_type='split')
    '''

project_data/WESAD/S10/S10_readme.txt
S10
project_data/WESAD/S10/S10_E4_Data.zip
Reading files
Calculating EDA features



                                   phasic_mean    phasic_std    phasic_min  \
datetime                                                                     
2017-07-25 07:06:08+00:00         1.137086e-02  2.334112e-02 -1.709645e-02   
2017-07-25 07:06:08.250000+00:00  1.137086e-02  2.334112e-02 -1.709645e-02   
2017-07-25 07:06:08.500000+00:00  1.137086e-02  2.334112e-02 -1.709645e-02   
2017-07-25 07:06:08.750000+00:00  1.139982e-02  2.332268e-02 -1.709645e-02   
2017-07-25 07:06:09+00:00         1.146601e-02  2.326762e-02 -1.709645e-02   
...                                        ...           ...           ...   
2017-07-25 08:59:48.500000+00:00  2.064050e-08  1.173070e-09  1.913305e-08   
2017-07-25 08:59:48.750000+00:00  2.025948e-08  8.833190e-10  1.913305e-08   
2017-07-25 08:59:49+00:00         1.991975e-08  6.310440e-10  1.913305e-08   
2017-07-25 08:59:49.250000+00:00  1.961967e-08  4.132487e-10  1.913305e-08   
2017-07-25 08:59:49.500000+00:00  1.935787e-08  2.248218e-10  1


                                  phasic_mean  phasic_std  phasic_min  \
datetime                                                                
2017-07-25 11:15:19+00:00            0.220541    0.315897   -0.002007   
2017-07-25 11:15:19.250000+00:00     0.220559    0.315884   -0.002007   
2017-07-25 11:15:19.500000+00:00     0.220576    0.315873   -0.002007   
2017-07-25 11:15:19.750000+00:00     0.220599    0.315856    0.002841   
2017-07-25 11:15:20+00:00            0.220560    0.315883    0.002841   
...                                       ...         ...         ...   
2017-07-25 13:02:58+00:00            0.000162    0.000033    0.000118   
2017-07-25 13:02:58.250000+00:00     0.000151    0.000025    0.000118   
2017-07-25 13:02:58.500000+00:00     0.000142    0.000019    0.000118   
2017-07-25 13:02:58.750000+00:00     0.000133    0.000013    0.000118   
2017-07-25 13:02:59+00:00            0.000125    0.000007    0.000118   

                                  phasic_max  pha


                                  phasic_mean  phasic_std  phasic_min  \
datetime                                                                
2017-08-08 11:14:07+00:00            0.228773    0.348083    0.000000   
2017-08-08 11:14:07.250000+00:00     0.229276    0.347839    0.000000   
2017-08-08 11:14:07.500000+00:00     0.229785    0.347592    0.002310   
2017-08-08 11:14:07.750000+00:00     0.230254    0.347372    0.002310   
2017-08-08 11:14:08+00:00            0.229881    0.347438    0.002310   
...                                       ...         ...         ...   
2017-08-08 13:08:28+00:00            0.042389    0.008974    0.030342   
2017-08-08 13:08:28.250000+00:00     0.039550    0.006947    0.030342   
2017-08-08 13:08:28.500000+00:00     0.036945    0.005139    0.030342   
2017-08-08 13:08:28.750000+00:00     0.034555    0.003515    0.030342   
2017-08-08 13:08:29+00:00            0.032359    0.002018    0.030342   

                                  phasic_max  pha


                                   phasic_mean    phasic_std    phasic_min  \
datetime                                                                     
2017-08-09 07:10:31+00:00         9.148106e-03  1.654181e-02 -1.736726e-02   
2017-08-09 07:10:31.250000+00:00  9.148139e-03  1.654179e-02 -1.736726e-02   
2017-08-09 07:10:31.500000+00:00  9.148168e-03  1.654177e-02 -1.736726e-02   
2017-08-09 07:10:31.750000+00:00  9.175905e-03  1.652082e-02 -1.736726e-02   
2017-08-09 07:10:32+00:00         9.240499e-03  1.645440e-02 -1.736726e-02   
...                                        ...           ...           ...   
2017-08-09 09:06:55+00:00         7.441395e-08  1.507902e-08  5.419683e-08   
2017-08-09 09:06:55.250000+00:00  6.963799e-08  1.166163e-08  5.419683e-08   
2017-08-09 09:06:55.500000+00:00  6.526238e-08  8.617567e-09  5.419683e-08   
2017-08-09 09:06:55.750000+00:00  6.125137e-08  5.887598e-09  5.419683e-08   
2017-08-09 09:06:56+00:00         5.757262e-08  3.375788e-09  5

...


                                  phasic_mean  phasic_std  phasic_min  \
datetime                                                                
2017-06-13 12:23:13+00:00            0.039206    0.070696   -0.011631   
2017-06-13 12:23:13.250000+00:00     0.039206    0.070696   -0.011631   
2017-06-13 12:23:13.500000+00:00     0.039206    0.070696   -0.011631   
2017-06-13 12:23:13.750000+00:00     0.039233    0.070679   -0.011631   
2017-06-13 12:23:14+00:00            0.039282    0.070648   -0.005069   
...                                       ...         ...         ...   
2017-06-13 14:29:02.500000+00:00     0.000829    0.000165    0.000605   
2017-06-13 14:29:02.750000+00:00     0.000777    0.000129    0.000605   
2017-06-13 14:29:03+00:00            0.000729    0.000096    0.000605   
2017-06-13 14:29:03.250000+00:00     0.000684    0.000066    0.000605   
2017-06-13 14:29:03.500000+00:00     0.000643    0.000038    0.000605   

                                  phasic_max  pha


                                  phasic_mean  phasic_std  phasic_min  \
datetime                                                                
2017-06-14 11:21:38+00:00            0.054378    0.087064   -0.008846   
2017-06-14 11:21:38.250000+00:00     0.054478    0.087016   -0.008846   
2017-06-14 11:21:38.500000+00:00     0.054582    0.086965   -0.008846   
2017-06-14 11:21:38.750000+00:00     0.054712    0.086898   -0.008846   
2017-06-14 11:21:39+00:00            0.054856    0.086822    0.000077   
...                                       ...         ...         ...   
2017-06-14 13:40:20+00:00            0.082172    0.004159    0.077333   
2017-06-14 13:40:20.250000+00:00     0.080735    0.002894    0.077333   
2017-06-14 13:40:20.500000+00:00     0.079536    0.001810    0.077333   
2017-06-14 13:40:20.750000+00:00     0.078645    0.001090    0.077333   
2017-06-14 13:40:21+00:00            0.077966    0.000633    0.077333   

                                  phasic_max  pha


                                  phasic_mean  phasic_std  phasic_min  \
datetime                                                                
2017-07-06 11:12:04+00:00            0.488881    0.766992   -0.006495   
2017-07-06 11:12:04.250000+00:00     0.489605    0.766612   -0.006495   
2017-07-06 11:12:04.500000+00:00     0.490313    0.766238   -0.006495   
2017-07-06 11:12:04.750000+00:00     0.491031    0.765852   -0.001066   
2017-07-06 11:12:05+00:00            0.491708    0.765489    0.000686   
...                                       ...         ...         ...   
2017-07-06 13:00:16+00:00            0.005571    0.004114    0.000368   
2017-07-06 13:00:16.250000+00:00     0.006612    0.003716    0.001509   
2017-07-06 13:00:16.500000+00:00     0.007888    0.003021    0.003759   
2017-07-06 13:00:16.750000+00:00     0.009264    0.002142    0.006584   
2017-07-06 13:00:17+00:00            0.010604    0.001223    0.009380   

                                  phasic_max  pha


                                   phasic_mean    phasic_std    phasic_min  \
datetime                                                                     
2017-07-10 11:11:40+00:00         6.413901e-02  7.523357e-02 -1.140862e-02   
2017-07-10 11:11:40.250000+00:00  6.479220e-02  7.535431e-02 -1.140862e-02   
2017-07-10 11:11:40.500000+00:00  6.539403e-02  7.541120e-02 -1.140862e-02   
2017-07-10 11:11:40.750000+00:00  6.597153e-02  7.539245e-02 -1.140862e-02   
2017-07-10 11:11:41+00:00         6.652395e-02  7.530912e-02 -6.571704e-03   
...                                        ...           ...           ...   
2017-07-10 13:02:16+00:00         1.080736e-08  1.869470e-10  1.054631e-08   
2017-07-10 13:02:16.250000+00:00  1.075118e-08  1.516593e-10  1.054631e-08   
2017-07-10 13:02:16.500000+00:00  1.069577e-08  1.157695e-10  1.054631e-08   
2017-07-10 13:02:16.750000+00:00  1.064233e-08  8.027306e-11  1.054631e-08   
2017-07-10 13:02:17+00:00         1.059210e-08  4.579024e-11  1


                                   phasic_mean    phasic_std    phasic_min  \
datetime                                                                     
2017-07-11 11:11:41+00:00         7.489995e-02  9.151289e-02 -6.985227e-03   
2017-07-11 11:11:41.250000+00:00  7.496216e-02  9.146701e-02 -6.985227e-03   
2017-07-11 11:11:41.500000+00:00  7.502052e-02  9.142362e-02 -6.985227e-03   
2017-07-11 11:11:41.750000+00:00  7.509742e-02  9.136369e-02 -6.985227e-03   
2017-07-11 11:11:42+00:00         7.517692e-02  9.130051e-02  3.544893e-04   
...                                        ...           ...           ...   
2017-07-11 12:55:27.500000+00:00  7.096246e-08  1.159822e-08  5.526111e-08   
2017-07-11 12:55:27.750000+00:00  6.732045e-08  9.046383e-09  5.526111e-08   
2017-07-11 12:55:28+00:00         6.394640e-08  6.736114e-09  5.526111e-08   
2017-07-11 12:55:28.250000+00:00  6.082240e-08  4.632788e-09  5.526111e-08   
2017-07-11 12:55:28.500000+00:00  5.793223e-08  2.671124e-09  5

# Get relevant features
